In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tsm.data_selector import data_subset_by_dict
import matplotlib.pyplot as plt
import numpy as np

In [3]:
train_data = pd.read_pickle('kaggle/input/ashrae-energy-prediction/train.pkl')

## Level 3A)
* 1 site - X buildings
* Meter reading: 0
* All features
--> Studying the missing values

In [6]:
level_3a = {'site_id': 9, 'meter': 0}
level_3a_cols = list(train_data)

In [7]:
data_3a = data_subset_by_dict(train_data, level_3a, level_3a_cols)

In [8]:
data_3a.head()

,building_id,timestamp,meter_reading,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,...,sea_level_pressure,wind_direction,wind_speed,dt_m,dt_w,dt_d,dt_hour,dt_day_week,dt_day_month,dt_week_month
1069,874,2016-01-01,0.0,6,181805,9999,99,99.0,99,99.0,...,9999.0,999,99.0,1,53,1,0,4,1,1
1071,875,2016-01-01,0.0,4,177708,9999,99,99.0,99,99.0,...,9999.0,999,99.0,1,53,1,0,4,1,1
1074,876,2016-01-01,0.0,0,55240,9999,99,99.0,99,99.0,...,9999.0,999,99.0,1,53,1,0,4,1,1
1076,877,2016-01-01,0.0,6,8004,9999,99,99.0,99,99.0,...,9999.0,999,99.0,1,53,1,0,4,1,1
1077,878,2016-01-01,0.0,4,41241,9999,99,99.0,99,99.0,...,9999.0,999,99.0,1,53,1,0,4,1,1


## Baseline 1) LGBM on hold out buildings

In [26]:
buildings = list(set(data_3a.building_id))

In [37]:
test_builds = [x for ix, x in enumerate(buildings) if ix % 5 == 0]

In [38]:
round(len(test_builds) / len(buildings) * 100, 3), '%'

(20.492, '%')

In [39]:
train_builds = [x for x in buildings if x not in test_builds]

In [41]:
data_3a_tr = data_3a[data_3a.building_id.isin(train_builds)]
data_3a_val = data_3a[data_3a.building_id.isin(test_builds[:int(len(test_builds)*0.25)])]
data_3a_ts = data_3a[data_3a.building_id.isin(test_builds[int(len(test_builds)*0.25):])]

In [42]:
import lightgbm as lgb

In [48]:
from tsm.eval_metrics import root_mean_squared_log_error

In [18]:
feat_cols = [x for x in list(train) if x not in ['meter_reading', 'timestamp']]

In [44]:
data_3a_tr_x = data_3a_tr[feat_cols]
data_3a_tr_y = data_3a_tr.meter_reading
data_3a_val_x = data_3a_val[feat_cols]
data_3a_val_y = data_3a_val.meter_reading
data_3a_ts_x = data_3a_ts[feat_cols]
data_3a_ts_y = data_3a_ts.meter_reading

In [43]:
SEED = 7

In [99]:
lgb_reg_params = {'objective':'regression',  'boosting_type':'gbdt', 'metric':'rmse',
                  'n_jobs':-1, 'learning_rate':0.07, 'num_leaves': 2**8, 'max_depth':-1,
                  'tree_learner':'serial', 'colsample_bytree': 0.7, 'subsample_freq':1,
                  'subsample':0.5, 'max_bin':255, 'verbose':1, 'seed': SEED, 'early_stopping_rounds':200} 
lgb_reg_params_2 = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'learning_rate': 0.5,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'bagging_freq' : 5
            }

#### Base

In [123]:
lgb_train = lgb.Dataset(data_3a_tr_x, data_3a_tr_y)
lgb_eval = lgb.Dataset(data_3a_val_x, data_3a_val_y)
gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                        early_stopping_rounds=20, verbose_eval = 20)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 165.681	valid_1's rmse: 99.0052
[40]	training's rmse: 149.75	valid_1's rmse: 95.5506
[60]	training's rmse: 142.258	valid_1's rmse: 93.4872
[80]	training's rmse: 136.743	valid_1's rmse: 92.3842
[100]	training's rmse: 131.498	valid_1's rmse: 91.9253
[120]	training's rmse: 127.599	valid_1's rmse: 91.5536
[140]	training's rmse: 124.153	valid_1's rmse: 91.3555
[160]	training's rmse: 121.366	valid_1's rmse: 91.2652
[180]	training's rmse: 118.28	valid_1's rmse: 91.1758
[200]	training's rmse: 115.549	valid_1's rmse: 91.0694
[220]	training's rmse: 112.81	valid_1's rmse: 91.1227
[240]	training's rmse: 110.942	valid_1's rmse: 90.9337
[260]	training's rmse: 108.458	valid_1's rmse: 90.9718
[280]	training's rmse: 105.658	valid_1's rmse: 90.9441
[300]	training's rmse: 103.634	valid_1's rmse: 90.9492
[320]	training's rmse: 101.463	valid_1's rmse: 90.8674
[340]	training's rmse: 99.7425	valid_1's rmse: 90.7437
[360]	trai

In [124]:
lgbm_hat = (gbm_regress.predict(data_3a_ts_x, num_iteration=gbm_regress.best_iteration))

In [125]:
root_mean_squared_log_error(lgbm_hat, data_3a_ts_y)

1.0442132734669594

In [19]:
train = data_3a

#### V1 - Simply take log of target

In [79]:
data_3a1_tr_y = np.log(data_3a_tr_y)
data_3a1_val_y = np.log(data_3a_val_y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [80]:
data_3a1_tr_x = data_3a_tr_x[np.isfinite(data_3a1_tr_y)]
data_3a1_tr_y = data_3a1_tr_y[np.isfinite(data_3a1_tr_y)]
data_3a1_val_x = data_3a_val_x[np.isfinite(data_3a1_val_y)]
data_3a1_val_y = data_3a1_val_y[np.isfinite(data_3a1_val_y)]

In [81]:
lgb_train = lgb.Dataset(data_3a1_tr_x, data_3a1_tr_y)
lgb_eval = lgb.Dataset(data_3a1_val_x, data_3a1_val_y)
gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                        early_stopping_rounds=20, verbose_eval = 20)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 0.35509	valid_1's rmse: 0.529354
[40]	training's rmse: 0.182267	valid_1's rmse: 0.52526
[60]	training's rmse: 0.149686	valid_1's rmse: 0.518928
[80]	training's rmse: 0.136748	valid_1's rmse: 0.515862
[100]	training's rmse: 0.129195	valid_1's rmse: 0.513888
[120]	training's rmse: 0.123949	valid_1's rmse: 0.513071
[140]	training's rmse: 0.12008	valid_1's rmse: 0.513156
[160]	training's rmse: 0.117099	valid_1's rmse: 0.512715
[180]	training's rmse: 0.114791	valid_1's rmse: 0.512592
[200]	training's rmse: 0.112721	valid_1's rmse: 0.512954
[220]	training's rmse: 0.11079	valid_1's rmse: 0.512793
[240]	training's rmse: 0.109261	valid_1's rmse: 0.512851
[260]	training's rmse: 0.107705	valid_1's rmse: 0.512624
[280]	training's rmse: 0.106267	valid_1's rmse: 0.512746
[300]	training's rmse: 0.105014	valid_1's rmse: 0.512767
[320]	training's rmse: 0.103954	valid_1's rmse: 0.51262
[340]	training's rmse: 0.102887	val

In [82]:
lgbm_hat = np.exp(gbm_regress.predict(data_3a_ts_x, num_iteration=gbm_regress.best_iteration))

In [83]:
root_mean_squared_log_error(lgbm_hat, data_3a_ts_y)

1.225877060708199

#### V2 - two steps model (0s and values)

In [84]:
data_3a2_tr_y = np.log(data_3a_tr_y)
data_3a2_val_y = np.log(data_3a_val_y)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


In [114]:
lgb_train = lgb.Dataset(data_3a_tr_x, data_3a2_tr_y > 0)
lgb_eval = lgb.Dataset(data_3a_val_x, data_3a2_val_y > 0)
class_params = {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'},
                'learning_rate': 0.1, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
                'bagging_freq' : 5}
gbm_class = lgb.train(class_params,
            lgb_train,
            num_boost_round=2000,
            valid_sets=(lgb_train, lgb_eval),
            early_stopping_rounds=20,
            verbose_eval = 20)
lgb_train = lgb.Dataset(data_3a_tr_x[data_3a2_tr_y > 0], data_3a2_tr_y[data_3a2_tr_y > 0])
lgb_eval = lgb.Dataset(data_3a_val_x[data_3a2_val_y > 0] , data_3a2_val_y[data_3a2_val_y > 0])
gbm_regress = lgb.train(lgb_reg_params,
            lgb_train,
            num_boost_round=2000,
            valid_sets=(lgb_train, lgb_eval),
            early_stopping_rounds=20,
            verbose_eval = 20)

Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.0308295	valid_1's binary_logloss: 0.0324629
[40]	training's binary_logloss: 0.01732	valid_1's binary_logloss: 0.0431441
Early stopping, best iteration is:
[39]	training's binary_logloss: 0.0175738	valid_1's binary_logloss: 0.023073
Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 0.354876	valid_1's rmse: 0.530515
[40]	training's rmse: 0.182497	valid_1's rmse: 0.524844
[60]	training's rmse: 0.149908	valid_1's rmse: 0.518112
[80]	training's rmse: 0.136598	valid_1's rmse: 0.516724
[100]	training's rmse: 0.1286	valid_1's rmse: 0.515128
[120]	training's rmse: 0.12373	valid_1's rmse: 0.514883
[140]	training's rmse: 0.119728	valid_1's rmse: 0.51461
[160]	training's rmse: 0.116941	valid_1's rmse: 0.514915
[180]	training's rmse: 0.114552	valid_1's rmse: 0.514947
[200]	training's rmse: 0.112598	valid_1's rmse: 0.515123
[220]	training's rmse: 0.110673	valid_1's rmse: 0

In [115]:
lgbm_hat = (gbm_class.predict(data_3a_ts_x, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(data_3a_ts_x, num_iteration=gbm_regress.best_iteration))

In [116]:
root_mean_squared_log_error(np.exp(lgbm_hat), data_3a_ts_y)

1.2510477087142877

### Base No 0s

In [120]:
lgb_train = lgb.Dataset(data_3a_tr_x[data_3a_tr_y > 0], data_3a_tr_y[data_3a_tr_y > 0])
lgb_eval = lgb.Dataset(data_3a_val_x[data_3a_val_y > 0], data_3a_val_y[data_3a_val_y > 0])
gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                        early_stopping_rounds=20, verbose_eval = 20)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 162.082	valid_1's rmse: 95.9119
[40]	training's rmse: 145.405	valid_1's rmse: 95.1372
[60]	training's rmse: 138.699	valid_1's rmse: 93.279
[80]	training's rmse: 132.478	valid_1's rmse: 92.4621
[100]	training's rmse: 127.441	valid_1's rmse: 91.9107
[120]	training's rmse: 123.408	valid_1's rmse: 91.8357
[140]	training's rmse: 118.953	valid_1's rmse: 91.9578
[160]	training's rmse: 115.316	valid_1's rmse: 91.919
[180]	training's rmse: 112.273	valid_1's rmse: 91.9883
[200]	training's rmse: 109.347	valid_1's rmse: 92.0166
[220]	training's rmse: 106.132	valid_1's rmse: 92.118
[240]	training's rmse: 103.194	valid_1's rmse: 92.258
[260]	training's rmse: 100.164	valid_1's rmse: 92.2789
[280]	training's rmse: 97.0306	valid_1's rmse: 92.4091
[300]	training's rmse: 94.5292	valid_1's rmse: 92.4188
Early stopping, best iteration is:
[119]	training's rmse: 123.628	valid_1's rmse: 91.835


In [121]:
lgbm_hat = (gbm_regress.predict(data_3a_ts_x, num_iteration=gbm_regress.best_iteration))

In [122]:
root_mean_squared_log_error(lgbm_hat, data_3a_ts_y)

1.2508481949525687

## Baseline 2) Random Forest on 5-fold cross validation

In [133]:
from sklearn.model_selection import KFold

In [129]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [130]:
rfr = RandomForestRegressor(max_depth=10, n_jobs=-1, n_estimators=11)

In [149]:
K = 5
kf = KFold(n_splits=K)

In [154]:
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix,:][feat_cols]
    data_3a2_tr_y = data_3a.iloc[tr_ix,:].meter_reading
    rfr.fit(data_3a2_tr_x, data_3a2_tr_y)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    nrmlse = root_mean_squared_log_error(rfr.predict(data_3a2_ts_x), data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)

Iteration done, nrmlse: 1.27656938494446
Iteration done, nrmlse: 0.766602725025143
Iteration done, nrmlse: 1.2128047921456724
Iteration done, nrmlse: 0.9400355848383093
Iteration done, nrmlse: 1.225469945224068


In [155]:
sum(nrmlses) / K

1.0842964864355307

In [160]:
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_3a2_tr_y = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    data_3a2_val_x = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_3a2_val_y = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    lgb_train = lgb.Dataset(data_3a2_tr_x, data_3a2_tr_y)
    lgb_eval = lgb.Dataset(data_3a2_val_x, data_3a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_3a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 160.466	valid_1's rmse: 129.627
[40]	training's rmse: 143.899	valid_1's rmse: 123.358
[60]	training's rmse: 136.853	valid_1's rmse: 124.195
[80]	training's rmse: 131.218	valid_1's rmse: 125.117
[100]	training's rmse: 126.613	valid_1's rmse: 125.788
[120]	training's rmse: 122.418	valid_1's rmse: 126.985
[140]	training's rmse: 118.099	valid_1's rmse: 128.082
[160]	training's rmse: 114.809	valid_1's rmse: 129.643
[180]	training's rmse: 112.109	valid_1's rmse: 130.377
[200]	training's rmse: 109.551	valid_1's rmse: 131.282
[220]	training's rmse: 107.265	valid_1's rmse: 132.371
[240]	training's rmse: 104.532	valid_1's rmse: 133.027
Early stopping, best iteration is:
[45]	training's rmse: 142.182	valid_1's rmse: 123.205


/Users/Ric/_personal/x_tsm/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


Iteration done, nrmlse: 1.2666714640419818
Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 174.159	valid_1's rmse: 130.792
[40]	training's rmse: 157.906	valid_1's rmse: 125.606
[60]	training's rmse: 149.999	valid_1's rmse: 126.847
[80]	training's rmse: 143.635	valid_1's rmse: 129.311
[100]	training's rmse: 139.235	valid_1's rmse: 130.483
[120]	training's rmse: 134.999	valid_1's rmse: 134.076
[140]	training's rmse: 130.505	valid_1's rmse: 133.819
[160]	training's rmse: 126.609	valid_1's rmse: 135.987
[180]	training's rmse: 123.779	valid_1's rmse: 136.058
[200]	training's rmse: 120.703	valid_1's rmse: 138.754
[220]	training's rmse: 118.284	valid_1's rmse: 139.353
Early stopping, best iteration is:
[31]	training's rmse: 163.156	valid_1's rmse: 125.227
Iteration done, nrmlse: 0.7978162658853369
Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 168.612	valid_1's rmse: 129.8
[40]	training's rmse: 152.919	valid_1's rmse: 12

In [161]:
print('FinalScore:', sum(nrmlses) / K)

0.9976603885071395

In [166]:
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_3a2_tr_y = np.log1p(data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading)
    data_3a2_val_x = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_3a2_val_y = np.log1p(data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading)
    lgb_train = lgb.Dataset(data_3a2_tr_x, data_3a2_tr_y)
    lgb_eval = lgb.Dataset(data_3a2_val_x, data_3a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_3a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(np.exp(lgbm_hat), data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 0.565292	valid_1's rmse: 0.584833
[40]	training's rmse: 0.320211	valid_1's rmse: 0.501368
[60]	training's rmse: 0.237014	valid_1's rmse: 0.477418
[80]	training's rmse: 0.201449	valid_1's rmse: 0.471083
[100]	training's rmse: 0.184674	valid_1's rmse: 0.466634
[120]	training's rmse: 0.174869	valid_1's rmse: 0.465181
[140]	training's rmse: 0.167523	valid_1's rmse: 0.464584
[160]	training's rmse: 0.162607	valid_1's rmse: 0.464419
[180]	training's rmse: 0.158674	valid_1's rmse: 0.464223
[200]	training's rmse: 0.155815	valid_1's rmse: 0.46498
[220]	training's rmse: 0.152788	valid_1's rmse: 0.466258
[240]	training's rmse: 0.149775	valid_1's rmse: 0.465904
[260]	training's rmse: 0.147445	valid_1's rmse: 0.46654
[280]	training's rmse: 0.145223	valid_1's rmse: 0.467333
[300]	training's rmse: 0.143208	valid_1's rmse: 0.467621
[320]	training's rmse: 0.141506	valid_1's rmse: 0.46894
[340]	training's rmse: 0.139937	v

In [167]:
# LogY No0s
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_3a2_tr_y = np.log1p(data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading)
    data_3a2_val_x = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_3a2_val_y = np.log1p(data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading)
    data_3a2_tr_x = data_3a2_tr_x[data_3a2_tr_y > 0]
    data_3a2_tr_y = data_3a2_tr_y[data_3a2_tr_y > 0]
    data_3a2_val_x = data_3a2_val_x[data_3a2_val_y > 0]
    data_3a2_val_y = data_3a2_val_y[data_3a2_val_y > 0]
    lgb_train = lgb.Dataset(data_3a2_tr_x, data_3a2_tr_y)
    lgb_eval = lgb.Dataset(data_3a2_val_x, data_3a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_3a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(np.exp(lgbm_hat), data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 0.336914	valid_1's rmse: 0.408581
[40]	training's rmse: 0.169896	valid_1's rmse: 0.274901
[60]	training's rmse: 0.135494	valid_1's rmse: 0.25342
[80]	training's rmse: 0.123442	valid_1's rmse: 0.24995
[100]	training's rmse: 0.115958	valid_1's rmse: 0.250243
[120]	training's rmse: 0.110749	valid_1's rmse: 0.251036
[140]	training's rmse: 0.107049	valid_1's rmse: 0.251718
[160]	training's rmse: 0.104148	valid_1's rmse: 0.25155
[180]	training's rmse: 0.102134	valid_1's rmse: 0.251799
[200]	training's rmse: 0.100375	valid_1's rmse: 0.252746
[220]	training's rmse: 0.0987583	valid_1's rmse: 0.253083
[240]	training's rmse: 0.0973732	valid_1's rmse: 0.253202
[260]	training's rmse: 0.0959241	valid_1's rmse: 0.253589
Early stopping, best iteration is:
[79]	training's rmse: 0.123806	valid_1's rmse: 0.249947
Iteration done, nrmlse: 1.3239670079300783
Training until validation scores don't improve for 200 rounds
[20]	

In [168]:
# 2-steps
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_3a2_tr_y = np.log1p(data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading)
    data_3a2_val_x = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_3a2_val_y = np.log1p(data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading)
    lgb_train = lgb.Dataset(data_3a2_tr_x, data_3a2_tr_y > 0)
    lgb_eval = lgb.Dataset(data_3a2_val_x, data_3a2_val_y > 0)
    class_params = {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': {'binary_logloss'},
                    'learning_rate': 0.1, 'feature_fraction': 0.8, 'bagging_fraction': 0.8,
                    'bagging_freq' : 5}
    gbm_class = lgb.train(class_params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=20,
                verbose_eval = 20)
    lgb_train = lgb.Dataset(data_3a2_tr_x[data_3a2_tr_y > 0], data_3a2_tr_y[data_3a2_tr_y > 0])
    lgb_eval = lgb.Dataset(data_3a2_val_x[data_3a2_val_y > 0] , data_3a2_val_y[data_3a2_val_y > 0])
    gbm_regress = lgb.train(lgb_reg_params,
                lgb_train,
                num_boost_round=2000,
                valid_sets=(lgb_train, lgb_eval),
                early_stopping_rounds=20,
                verbose_eval = 20)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_class.predict(data_3a2_ts_x, num_iteration=gbm_class.best_iteration) > .5) *\
    (gbm_regress.predict(data_3a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(np.exp(lgbm_hat), data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)
print('FinalScore:', sum(nrmlses) / K)

Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.0196934	valid_1's binary_logloss: 0.0417943
Early stopping, best iteration is:
[19]	training's binary_logloss: 0.020766	valid_1's binary_logloss: 0.0417425
Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 0.336914	valid_1's rmse: 0.408581
[40]	training's rmse: 0.169896	valid_1's rmse: 0.274901
[60]	training's rmse: 0.135494	valid_1's rmse: 0.25342
[80]	training's rmse: 0.123442	valid_1's rmse: 0.24995
[100]	training's rmse: 0.115958	valid_1's rmse: 0.250243
[120]	training's rmse: 0.110749	valid_1's rmse: 0.251036
[140]	training's rmse: 0.107049	valid_1's rmse: 0.251718
[160]	training's rmse: 0.104148	valid_1's rmse: 0.25155
[180]	training's rmse: 0.102134	valid_1's rmse: 0.251799
[200]	training's rmse: 0.100375	valid_1's rmse: 0.252746
[220]	training's rmse: 0.0987583	valid_1's rmse: 0.253083
[240]	training's rmse: 0.0973732	valid_1's rmse: 0.253202
[260]	tr

In [169]:
nrmlses = []
for tr_ix, ts_ix in kf.split(data_3a):
    data_3a2_tr_x = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:][feat_cols]
    data_3a2_tr_y = data_3a.iloc[tr_ix[:int(len(tr_ix)*.9)],:].meter_reading
    data_3a2_val_x = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:][feat_cols]
    data_3a2_val_y = data_3a.iloc[tr_ix[int(len(tr_ix)*.9):],:].meter_reading
    data_3a2_tr_x = data_3a2_tr_x[data_3a2_tr_y > 0]
    data_3a2_tr_y = data_3a2_tr_y[data_3a2_tr_y > 0]
    data_3a2_val_x = data_3a2_val_x[data_3a2_val_y > 0]
    data_3a2_val_y = data_3a2_val_y[data_3a2_val_y > 0]
    lgb_train = lgb.Dataset(data_3a2_tr_x, data_3a2_tr_y)
    lgb_eval = lgb.Dataset(data_3a2_val_x, data_3a2_val_y)
    gbm_regress = lgb.train(lgb_reg_params, lgb_train, num_boost_round=2000, valid_sets=(lgb_train, lgb_eval),
                            early_stopping_rounds=20, verbose_eval = 20)
    data_3a2_ts_x = data_3a.iloc[ts_ix,:][feat_cols]
    data_3a2_ts_y = data_3a.iloc[ts_ix,:].meter_reading
    lgbm_hat = (gbm_regress.predict(data_3a2_ts_x, num_iteration=gbm_regress.best_iteration))
    nrmlse = root_mean_squared_log_error(lgbm_hat, data_3a2_ts_y)
    print('Iteration done, nrmlse:', nrmlse)
    nrmlses.append(nrmlse)

Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 157.8	valid_1's rmse: 124.971
[40]	training's rmse: 140.907	valid_1's rmse: 116.844
[60]	training's rmse: 133.401	valid_1's rmse: 117.381
[80]	training's rmse: 125.511	valid_1's rmse: 119.977
[100]	training's rmse: 119.693	valid_1's rmse: 119.185
[120]	training's rmse: 114.772	valid_1's rmse: 120.887
[140]	training's rmse: 109.899	valid_1's rmse: 121.247
[160]	training's rmse: 106.235	valid_1's rmse: 122.154
[180]	training's rmse: 103.113	valid_1's rmse: 123.782
[200]	training's rmse: 99.1186	valid_1's rmse: 124.137
[220]	training's rmse: 96.026	valid_1's rmse: 125.503
[240]	training's rmse: 92.9901	valid_1's rmse: 125.953
Early stopping, best iteration is:
[42]	training's rmse: 140.158	valid_1's rmse: 116.447
Iteration done, nrmlse: 1.3959323989867758
Training until validation scores don't improve for 200 rounds
[20]	training's rmse: 171.452	valid_1's rmse: 121.304
[40]	training's rmse: 154.7	valid_1's

/Users/Ric/_personal/x_tsm/tsm/eval_metrics.py:5: RuntimeWarning: invalid value encountered in log
  return np.sqrt(np.sum(np.power(np.log(predicted + 1) - np.log(actual + 1), 2)) / len(actual))


In [170]:
sum(nrmlses) / K

1.0593330507889636